# Fine-Tuning GPT-4 with OpenAI API

In [2]:

# Install necessary libraries
import jsonlines
import json
import openai
import os
import pandas as pd
from pprint import pprint
from openai import OpenAI

## Set Up API Key

In [3]:
client = OpenAI(
    api_key="API KEY OPENAI",
)

## Load Fine-Tuning Data

In [4]:
# Load and validate your .jsonl file for conversational fine-tuning
training_file_name = "cars_data_training.jsonl"

validation_file_name = "cars_data_validation.jsonl"

## Upload File to OpenAI for Fine-Tuning

In [4]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-eS1WYcMl3ELA03QEPPQfyn5n
Validation file ID: file-yl1pCUSxjz09v7ttQpFWgjWt


## Start Fine-Tuning Job

In [5]:
# Create the fine-tuning job
MODEL = "gpt-4o-2024-08-06" # BISA DIMODIFIKASI DENGAN MODEL YANG LAIN SESUAI KEBUTUHAN

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="car-product", # BISA DIMODIFIKASI SESUAI DOMAIN REKOMENDASI
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-SRgkf44mbpLHqAitPDFOVjq6
Status: validating_files


In [6]:
# Check job status
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-SRgkf44mbpLHqAitPDFOVjq6
Status: validating_files
Trained Tokens: None


In [7]:
# Track the progress
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-SRgkf44mbpLHqAitPDFOVjq6
Validating training file: file-eS1WYcMl3ELA03QEPPQfyn5n and validation file: file-yl1pCUSxjz09v7ttQpFWgjWt


In [9]:
# Fine-tuned model ID
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-4o-2024-08-06:project-ta:car-product:AWHjeoF0
